# Activate Environment and import necessary modules

In [2]:
!source /Users/amirejibiilia/Desktop/Thesis/new_venv/bin/activate

In [3]:
import time
import numpy as np
import pandas as pd 

#plotting
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")

from sklearn.model_selection import KFold, GridSearchCV, cross_val_score, train_test_split # Model evaluation
from sklearn.preprocessing import MinMaxScaler # Data normalization

from xgboost import XGBRegressor, DMatrix, plot_importance # XGBoost

import tensorflow as tf
from tensorflow.keras.layers import Input, Dropout, Dense, LSTM, TimeDistributed, RepeatVector
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers

import matplotlib.dates as mdates
import optuna
from sklearn.metrics import mean_squared_error

2024-06-20 22:58:49.850917: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
# data import
df_train = pd.read_excel("/Users/amirejibiilia/Desktop/Thesis/version_4.xlsx", index_col = 0) 
cols_list = list(df_train.columns) 

In [5]:
df_train.head().round(2)

,HousingIndex,Capitalisation,ExchangeRate,InterestRateOnDeposits,CPI,HouseAffordabilityIndex,RentIndex,MortgageRate
Date,,,,,,,,
2005-01-01,163.43,8.53,1.82,7.66,102.09,40.0,111.90,0.16
2005-02-01,159.72,9.56,1.83,7.49,100.32,40.0,122.59,0.16
2005-03-01,178.20,8.62,1.84,7.04,100.59,40.0,123.29,0.16
2005-04-01,183.23,8.81,1.83,7.08,100.42,40.0,129.64,0.16
2005-05-01,175.51,9.45,1.83,7.58,98.71,40.0,133.09,0.16


# LSTM Autoencoder - Anomaly Detection Approach

In [43]:
import time
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

#plotting
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")

from sklearn.model_selection import KFold, GridSearchCV, cross_val_score, train_test_split # Model evaluation
from sklearn.preprocessing import MinMaxScaler # Data normalization

# pip install tensorflow==2
import tensorflow as tf
from tensorflow.keras.layers import Input, Dropout, Dense, LSTM, TimeDistributed, RepeatVector
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers

## LSTM - Data Preparation

In [44]:
# # Data normalization
scaler = MinMaxScaler(feature_range=(0, 1))
X_train = scaler.fit_transform(df_train[:-15])
X_test = scaler.transform(df_train[-15:])

In [45]:
# Reshape for LSTM
X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

## Hyperparameters Tuning of LSTM

In [46]:
# List to store hyperparameters, validation loss, and trial histories of each trial
all_trial_params = []
all_trial_val_losses = []
all_trial_histories = [] 

In [47]:
# Objective Function

def objective(trial):
    global all_trial_params, all_trial_val_losses, all_trial_histories  # Access the global lists
    
    # Define the hyperparameters to optimize
    lstm1_units = trial.suggest_int('lstm1_units', 15, 20)
    lstm2_units = trial.suggest_int('lstm2_units', 15, 15)
    regularizer_l2 = trial.suggest_uniform('regularizer_l2', 0.0, 0.1)
    
    # Build the model
    inputs = Input(shape=(X_train.shape[1], X_train.shape[2]))
    L1 = LSTM(lstm1_units, activation='relu', return_sequences=True, 
              kernel_regularizer=regularizers.l2(regularizer_l2))(inputs)
    L2 = LSTM(lstm2_units, activation='relu', return_sequences=False)(L1)
    L3 = RepeatVector(X_train.shape[1])(L2)
    L4 = LSTM(lstm2_units, activation='relu', return_sequences=True)(L3)
    L5 = LSTM(lstm1_units, activation='relu', return_sequences=True)(L4)
    output = TimeDistributed(Dense(X_train.shape[2]))(L5)    
    model = Model(inputs=inputs, outputs=output)
    
    # Compile the model
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)  # Fixed learning rate
    model.compile(optimizer=optimizer, loss='mean_squared_error')
    
    # Train-validation split based on time series data
    split_index = int(0.8 * len(X_train))  # Use first 80% for training
    X_train_split = X_train[:split_index]
    X_val_split = X_train[split_index:]
    
    # Train the model
    trial_history = model.fit(X_train_split, X_train_split, epochs=100, batch_size=10, 
                              validation_data=(X_val_split, X_val_split))
    
    # Get the validation loss
    val_loss = trial_history.history['val_loss'][-1]
    
    # Store hyperparameters, validation loss, and trial history
    all_trial_params.append((lstm1_units, lstm2_units))
    all_trial_val_losses.append(val_loss)
    all_trial_histories.append(trial_history)  # Store the history
    
    return val_loss

# Create a study object and optimize the objective function
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=200)

# Get the best hyperparameters
best_params = study.best_params
print("Best hyperparameters:", best_params)

[I 2024-06-13 22:44:24,763] A new study created in memory with name: no-name-78899ed0-fd92-4286-bcdc-f958b646b576
/var/folders/fb/0f4v33f515vfq6921ss69f180000gn/T/ipykernel_66441/1225587742.py:9: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  regularizer_l2 = trial.suggest_uniform('regularizer_l2', 0.0, 0.1)
2024-06-13 22:44:24.783465: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/100
18/18 [==============================] - 5s 46ms/step - loss: 0.3390 - val_loss: 0.4741
Epoch 2/100
18/18 [==============================] - 0s 7ms/step - loss: 0.2897 - val_loss: 0.4173
Epoch 3/100
18/18 [==============================] - 0s 7ms/step - loss: 0.2440 - val_loss: 0.3607
Epoch 4/100
18/18 [==============================] - 0s 7ms/step - loss: 0.1995 - val_loss: 0.2987
Epoch 5/100
18/18 [==============================] - 0s 7ms/step - loss: 0.1529 - val_loss: 0.2215
Epoch 6/100
18/18 [==============================] - 0s 8ms/step - loss: 0.1037 - val_loss: 0.1283
Epoch 7/100
18/18 [==============================] - 0s 7ms/step - loss: 0.0774 - val_loss: 0.1335
Epoch 8/100
18/18 [==============================] - 0s 6ms/step - loss: 0.0648 - val_loss: 0.1386
Epoch 9/100
18/18 [==============================] - 0s 7ms/step - loss: 0.0579 - val_loss: 0.1476
Epoch 10/100
18/18 [==============================] - 0s 6ms/step - loss: 0.0537 - val_loss: 0.1519
Epoch 11

Epoch 83/100
18/18 [==============================] - 0s 7ms/step - loss: 0.0104 - val_loss: 0.0463
Epoch 84/100
18/18 [==============================] - 0s 7ms/step - loss: 0.0105 - val_loss: 0.0469
Epoch 85/100
18/18 [==============================] - 0s 6ms/step - loss: 0.0105 - val_loss: 0.0472
Epoch 86/100
18/18 [==============================] - 0s 6ms/step - loss: 0.0105 - val_loss: 0.0485
Epoch 87/100
18/18 [==============================] - 0s 6ms/step - loss: 0.0104 - val_loss: 0.0441
Epoch 88/100
18/18 [==============================] - 0s 6ms/step - loss: 0.0103 - val_loss: 0.0456
Epoch 89/100
18/18 [==============================] - 0s 6ms/step - loss: 0.0103 - val_loss: 0.0465
Epoch 90/100
18/18 [==============================] - 0s 6ms/step - loss: 0.0105 - val_loss: 0.0467
Epoch 91/100
18/18 [==============================] - 0s 6ms/step - loss: 0.0104 - val_loss: 0.0458
Epoch 92/100
18/18 [==============================] - 0s 6ms/step - loss: 0.0103 - val_loss: 0.0437


[I 2024-06-13 22:44:42,216] Trial 0 finished with value: 0.0439583882689476 and parameters: {'lstm1_units': 30, 'lstm2_units': 16, 'regularizer_l2': 0.009516092640794095}. Best is trial 0 with value: 0.0439583882689476.


Epoch 1/100
18/18 [==============================] - 5s 41ms/step - loss: 0.2184 - val_loss: 0.3718
Epoch 2/100
18/18 [==============================] - 0s 7ms/step - loss: 0.1980 - val_loss: 0.3434
Epoch 3/100
18/18 [==============================] - 0s 6ms/step - loss: 0.1767 - val_loss: 0.3126
Epoch 4/100
18/18 [==============================] - 0s 6ms/step - loss: 0.1542 - val_loss: 0.2786
Epoch 5/100
18/18 [==============================] - 0s 6ms/step - loss: 0.1305 - val_loss: 0.2408
Epoch 6/100
18/18 [==============================] - 0s 6ms/step - loss: 0.1055 - val_loss: 0.1991
Epoch 7/100
18/18 [==============================] - 0s 6ms/step - loss: 0.0792 - val_loss: 0.1427
Epoch 8/100
18/18 [==============================] - 0s 6ms/step - loss: 0.0557 - val_loss: 0.1062
Epoch 9/100
18/18 [==============================] - 0s 6ms/step - loss: 0.0494 - val_loss: 0.1243
Epoch 10/100
18/18 [==============================] - 0s 7ms/step - loss: 0.0468 - val_loss: 0.1318
Epoch 11

...

[I 2024-06-13 23:49:43,729] Trial 198 finished with value: 0.02888726070523262 and parameters: {'lstm1_units': 26, 'lstm2_units': 13, 'regularizer_l2': 2.4748321306262616e-05}. Best is trial 37 with value: 0.023434150964021683.


Epoch 1/100
18/18 [==============================] - 5s 43ms/step - loss: 0.2571 - val_loss: 0.4072
Epoch 2/100
18/18 [==============================] - 0s 8ms/step - loss: 0.2290 - val_loss: 0.3732
Epoch 3/100
18/18 [==============================] - 0s 7ms/step - loss: 0.2017 - val_loss: 0.3388
Epoch 4/100
18/18 [==============================] - 0s 7ms/step - loss: 0.1739 - val_loss: 0.3015
Epoch 5/100
18/18 [==============================] - 0s 7ms/step - loss: 0.1450 - val_loss: 0.2598
Epoch 6/100
18/18 [==============================] - 0s 7ms/step - loss: 0.1144 - val_loss: 0.2126
Epoch 7/100
18/18 [==============================] - 0s 8ms/step - loss: 0.0835 - val_loss: 0.1596
Epoch 8/100
18/18 [==============================] - 0s 8ms/step - loss: 0.0631 - val_loss: 0.1392
Epoch 9/100
18/18 [==============================] - 0s 8ms/step - loss: 0.0537 - val_loss: 0.1454
Epoch 10/100
18/18 [==============================] - 0s 7ms/step - loss: 0.0492 - val_loss: 0.1434
Epoch 11

Epoch 83/100
18/18 [==============================] - 0s 8ms/step - loss: 0.0110 - val_loss: 0.0448
Epoch 84/100
18/18 [==============================] - 0s 7ms/step - loss: 0.0109 - val_loss: 0.0458
Epoch 85/100
18/18 [==============================] - 0s 7ms/step - loss: 0.0109 - val_loss: 0.0462
Epoch 86/100
18/18 [==============================] - 0s 8ms/step - loss: 0.0108 - val_loss: 0.0471
Epoch 87/100
18/18 [==============================] - 0s 7ms/step - loss: 0.0109 - val_loss: 0.0471
Epoch 88/100
18/18 [==============================] - 0s 7ms/step - loss: 0.0108 - val_loss: 0.0448
Epoch 89/100
18/18 [==============================] - 0s 8ms/step - loss: 0.0108 - val_loss: 0.0461
Epoch 90/100
18/18 [==============================] - 0s 7ms/step - loss: 0.0108 - val_loss: 0.0455
Epoch 91/100
18/18 [==============================] - 0s 8ms/step - loss: 0.0109 - val_loss: 0.0441
Epoch 92/100
18/18 [==============================] - 0s 8ms/step - loss: 0.0110 - val_loss: 0.0458


[I 2024-06-13 23:50:02,234] Trial 199 finished with value: 0.045424483716487885 and parameters: {'lstm1_units': 29, 'lstm2_units': 13, 'regularizer_l2': 0.0035110744646198273}. Best is trial 37 with value: 0.023434150964021683.


Best hyperparameters: {'lstm1_units': 19, 'lstm2_units': 16, 'regularizer_l2': 0.00037312915689845716}


In [48]:
#Best Parameters Themselves!
best_params_df = pd.DataFrame([best_params])
best_params_df

,lstm1_units,lstm2_units,regularizer_l2
0,19,16,0.000373
